# Get list of cited DOIs to analyze

In [1]:
import os
import subprocess
from bs4 import BeautifulSoup
import glob

import pandas as pd

# 1. List of DOIs
DOI_list = ["10.1126/science.1179052"]
out_csv = '/project/elife/data/data2v/DOI_cited_science_1179052_retracted.csv'

# 2. Reading list of DOIs from file
#file_to_read = '/project/elife/data/data2v/DOI_cited_1000v2.csv' #file with cited DOIs
#out_csv = '/project/elife/data/data2v/DOI_cited_1000v2_papers.csv'

#df_info_doi = pd.read_csv(file_to_read, sep='\t', header = None, encoding='utf-8')
#DOI_list = df_info_doi[1].tolist()



"\n\n# citations 1000-1100\nDOI_list = ['10.1126/science.1235122','10.7326/0003-4819-121-12-199412150-00009','10.1111/j.1538-7836.2006.01753.x','10.1056/NEJMoa021641','10.1007/978-3-642-69746-3_2', '10.1126/science.8197455', '10.1007/978-1-4613-4446-9', '10.1073/pnas.0135058100', '10.1038/5007','10.1111/j.1468-5914.1990.tb00174.x']\n\n\n# citations 2000-2100\nDOI_list = ['10.1183/09031936.05.00034805', '10.1207/S15328007SEM0902_5','10.1070/PU1968v010n04ABEH003699', '10.1038/nmeth.1701', '10.1016/S0032-9592(98)00112-5','10.1021/ac60131a045', '10.1073/pnas.1530509100', '10.1126/science.279.5350.548','10.1126/science.1058040', '10.1126/science.1209688']\n\n#citations 3000-3100\nDOI_list = ['10.1016/j.carbon.2007.02.034', '10.1113/jphysiol.1952.sp004764', '10.1093/geront/9.3_Part_1.179', '10.1021/cr00033a004', '10.1056/NEJM199901143400207', '10.1038/451652a','10.1137/S003614450342480', '10.1093/jnci/92.3.205', '10.1002/jcc.20289', '10.1186/gb-2002-3-7-research0034']\n\n#citations 4000-4100

# Find papers in FTP_PUBMED citing these DOIs

In [2]:
import os
import subprocess
from bs4 import BeautifulSoup
import glob

import pandas as pd


os.chdir('/project/FTP_PUBMED_papers')
os.getcwd()

# out_csv = '/project/elife/data/data2v/DOI_cited_1000v2_papers.csv'

for DOI in DOI_list:
    print(DOI)
    DOI_cited_in  = " "
    for filename in glob.iglob('/project/FTP_PUBMED_papers/*'):
        try:
            print('%s' % filename)
            just_filename = filename.replace('/project/FTP_PUBMED_papers/', ' ')
            bashCommand = "zgrep -a " + DOI + " " + just_filename
            DOI_cited_in_bytes = subprocess.check_output(bashCommand, shell=True)
            DOI_cited_in = DOI_cited_in + " " + DOI_cited_in_bytes.decode("utf-8") # conver bytes into strings

        except subprocess.CalledProcessError as e:
            print('zgrep returned non-zero exit status 1 when searching at %s' % filename)
            print("Subprocess output:", e.output)

     #   print("Saving data in file")
     #   df = pd.DataFrame({"DOI":DOI, "articles_citing": DOI_cited_in}, [0])
     #   df.to_csv(out_csv,header=False, mode='a', sep='\t', encoding='utf-8')
     #   print("Data saved")

    
    list_articles_citing = []
    DOI_cited_in_BS = BeautifulSoup(DOI_cited_in, "lxml")
    for article in DOI_cited_in_BS.prettify().split('</article>'):
        article_BS = BeautifulSoup(article, "lxml")# Not all the articles start or finish with this tag.
        for article2 in article_BS.prettify().split('</back>'): #('<article article-type'): #In some cases the seaparation between 2 articles i<\ref> <article article-type> 
            if (DOI in article2) and (len(str(article2)) > 800): #To remove spliting of some end-tags
                list_articles_citing.append(article2)
    
    df2 = pd.DataFrame({})
    for article in list_articles_citing:
        df2 = df2.append([{'DOI': DOI, 'articles_citing':article}])
    
    df2.to_csv(out_csv,header=False, mode='a', sep='\t', encoding='utf-8')
    print("Data saved")

    


10.1126/science.1179052
/project/FTP_PUBMED_papers/comm_use.C-H.xml.tar.gz
/project/FTP_PUBMED_papers/non_comm_use.O-Z.xml.tar.gz
/project/FTP_PUBMED_papers/comm_use.A-B.xml.tar.gz
/project/FTP_PUBMED_papers/comm_use.I-N.xml.tar.gz
/project/FTP_PUBMED_papers/non_comm_use.A-B.xml.tar.gz
zgrep returned non-zero exit status 1 when searching at /project/FTP_PUBMED_papers/non_comm_use.A-B.xml.tar.gz
Subprocess output: b''
/project/FTP_PUBMED_papers/non_comm_use.C-H.xml.tar.gz
/project/FTP_PUBMED_papers/non_comm_use.I-N.xml.tar.gz
zgrep returned non-zero exit status 1 when searching at /project/FTP_PUBMED_papers/non_comm_use.I-N.xml.tar.gz
Subprocess output: b''
/project/FTP_PUBMED_papers/comm_use.O-Z.xml.tar.gz
Data saved


# Checking files

In [9]:
import pandas as pd
file_csv = '/project/elife/data/DOI_cited_5000.csv'
df1000 = pd.read_csv(file_csv, sep='\t', header = None, encoding='utf-8', names = ['DOI_cites', 'article_citing'])
df1000.reset_index()
df1000.describe()

,DOI_cites,article_citing
count,804,804
unique,3,804
top,10.1093/nar/28.1.235,\n Beilstein_J_Nanotechnol/PMC4979881.nxml ...
freq,692,1


In [10]:
for DOI in df1000.DOI_cites.unique():
    print(df1000[df1000['DOI_cites'] == DOI].shape, '\t\t', DOI)

(692, 2) 		 10.1093/nar/28.1.235
(98, 2) 		 10.1063/1.448118
(14, 2) 		 10.1103/PhysRevB.46.6671


# Prepocessing data. Remove random spaces and store in a new file.

In [3]:
import sys
sys.path.insert(0, "/project/elife/modules")
import re
import bs_preprocess
import pandas as pd

# file to read 
#file_csv = '/project/elife/data/data2v/DOI_cited_1000v2_papers.csv'
file_csv = '/project/elife/data/data2v/DOI_cited_science_1179052_retracted.csv'

df1000 = pd.read_csv(file_csv, sep='\t', header = None, encoding='utf-8', names = ['DOI_cited', 'article_citing'])
df1000.reset_index()

# preprocess data
df1000['article_citing'] = df1000['article_citing'].apply(bs_preprocess.bs_preprocess)    

# save data
out_csv = '/project/elife/data/data2v/DOI_cited_science_1179052_retracted_prep.csv'
#out_csv = '/project/elife/data/data2v/DOI_cited_1000v2_papers_preprocess.csv'

df1000.to_csv(out_csv,header=False, mode='a', sep='\t', encoding='utf-8')

